In [1]:
from dotenv import load_dotenv
from agents import Agent, Runner, trace, function_tool
from openai.types.responses import ResponseTextDeltaEvent
from typing import Dict
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
import asyncio

In [2]:
load_dotenv(override=True)

True

In [3]:
# Let's just check emails are working for you

def send_test_email():
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sabapathyk@outlook.com")  # Change to your verified sender
    to_email = To("sabapathy.ksp@gmail.com")  # Change to your recipient
    content = Content("text/plain", "This is an important test email")
    mail = Mail(from_email, to_email, "Test email", content).get()
    response = sg.client.mail.send.post(request_body=mail)
    print(response.status_code)

send_test_email()

202


### Part 1:Agent Workflow 

In [4]:
instructions1 = "You are a sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write professional, serious cold emails."

instructions2 = "You are a humorous, engaging sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write witty, engaging cold emails that are likely to get a response."

instructions3 = "You are a busy sales agent working for ComplAI, \
a company that provides a SaaS tool for ensuring SOC2 compliance and preparing for audits, powered by AI. \
You write concise, to the point cold emails."

In [5]:
sales_agent1 = Agent(name="Professional Sales Agent",
                     instructions=instructions1,model="gpt-4o-mini")

sales_agent2 = Agent(name="Engaging Sales Agent",
                     instructions=instructions2,model="gpt-4o-mini")

sales_agent3 = Agent(name="Busy Sales Agent",
                     instructions=instructions3,model="gpt-4o-mini")

In [6]:
result = Runner.run_streamed(sales_agent1, input="Write a cold sales email")
async for event in result.stream_events():
    if event.type =="raw_response_event" and isinstance(event.data, ResponseTextDeltaEvent):
        print(event.data.delta, end='', flush=True)

Subject: Simplifying Your SOC 2 Compliance Process

Hi [Recipient's Name],

I hope this message finds you well. My name is [Your Name], and I’m reaching out from ComplAI, a pioneering SaaS solution dedicated to streamlining SOC 2 compliance and audit preparation.

In today’s fast-paced environment, maintaining compliance can be both time-consuming and complex. Our AI-powered platform automates critical tasks, reduces human error, and accelerates the entire compliance process. With ComplAI, your team can focus on core business objectives while ensuring a robust compliance framework.

Key benefits of our solution include:

- **Automation of essential compliance tasks**: Save time and resources with our intelligent workflows.
- **Real-time tracking**: Stay updated on your compliance posture and audit readiness at all times.
- **Intuitive insights and reporting**: Make informed decisions with easy-to-understand analytics.

I would love to explore how ComplAI can support [Recipient's Compan

In [7]:
message = "Write a cold sales email"

with trace("Parallel cold email"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message)
    )

outputs = [result.final_output for result in results]

for output in outputs:
    print(output + "\n\n")

Subject: Streamline Your SOC2 Compliance with ComplAI

Dear [Recipient's Name],

I hope this message finds you well. I am reaching out to introduce you to ComplAI, an innovative SaaS solution designed to simplify SOC2 compliance and audit preparations through the power of artificial intelligence.

As you know, maintaining compliance can be a complex and time-consuming process. ComplAI automates crucial aspects of SOC2 compliance, ensuring that your organization not only meets regulatory requirements but also enhances trust with your clients.

Here are a few ways ComplAI can benefit your organization:

1. **Automated Documentation:** Reduce the manual workload associated with creating and maintaining compliance documents.
2. **Real-Time Monitoring:** Gain insights into your compliance status with live updates and analytics.
3. **Seamless Audit Preparation:** Simplify the audit process from start to finish with streamlined workflows.

I would love the opportunity to discuss how ComplAI c

In [9]:
sales_picker = Agent(
    name="sales_picker",
    instructions="You are an expert sales email evaluator. \
Imagine your a customer and pick the one you are most likely respond to. \
Do not give an explanation; reply with the selected email only.",
model="gpt-4o-mini")


In [10]:
message = "Write a cold sales email"

with trace("Selection from sales people"):
    results = await asyncio.gather(
        Runner.run(sales_agent1, message),
        Runner.run(sales_agent2, message),
        Runner.run(sales_agent3, message),
    )

    outputs = [result.final_output for result in results]

    emails = "Cold sales emails:\n\n" + "\n\nEmail:\n\n".join(outputs)

    best = await Runner.run(sales_picker, emails)

    print(f"Best sales email:{best}")

Best sales email:RunResult:
- Last agent: Agent(name="sales_picker", ...)
- Final output (str):
    Subject: Don’t Let Compliance Be Your Next Headache!
    
    Hi [Recipient's Name],
    
    I hope this email finds you thriving and not buried under a mountain of compliance paperwork! 🤯 
    
    I’m [Your Name] from ComplAI, where we help companies like yours turn the daunting task of SOC2 compliance into a walk in the park (with a cool drink in hand, of course). 🍹 
    
    Imagine a world where audits are less about panic and more about confidence. Our AI-powered SaaS tool is like your personal compliance superhero, swooping in to streamline the process, automate the nitty-gritty, and—dare I say—actually make compliance enjoyable.
    
    So, if you're ready to kick those compliance headaches to the curb, let’s chat! I promise to bring the humor, and you can bring your toughest audit questions. 
    
    Looking forward to a future where compliance is a breeze!
    
    Best,  
 